## Trajectory analysis of GPS data - Kelowna (by SovereignAI) - Single day and multiple trajectory

In [ ]:
import requests
import json
import time
import datetime
from urllib.request import urlopen
import requests
import warnings
from copy import copy

import numpy as np
import pandas as pd
import seaborn as sns
import osmnx as ox
import networkx as nx
import geopandas as gpd

from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pyproj import CRS
import os
import movingpandas as mpd
from shapely.geometry import Point

# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Read csv
'''
kelowna_gps = pd.read_csv('C:/data/gps_sai/Kelowna_Redo.csv')
'''
kelowna_gps_01Aug = pd.read_csv('C:/data/gps_sai/kelowna_gps_parts/kelowna_gps_2023-08-01.csv')

# Display
kelowna_gps_01Aug

In [ ]:
def convert_df_to_gdf_v1(df, crs_no):
    # Create a geometry column from latitude and longitude and create gdf for Halifax
    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    
    # Set the coordinate reference system (CRS) if needed
    gdf.set_crs(epsg=crs_no, inplace=True)  # WGS 84
    
    # Display
    return gdf

In [ ]:
# CRS for the gdf
crs = 4326

kelowna_gps_01Aug = convert_df_to_gdf_v1(kelowna_gps_01Aug, crs)
kelowna_gps_01Aug

In [ ]:
def drop_less_freq_ids(df, col_name, freq):
    # Drop duplicates
    df = df.drop_duplicates()

    # Count occurrences of each registrationID
    registration_counts = df[col_name].value_counts()

    # Identify IDs that occur less than 4 times
    less_freq_ids = registration_counts[registration_counts < freq].index

    # Calculate the number of such IDs
    num_less_freq_ids = len(less_freq_ids)

    # Drop rows with registrationIDs that occur only once
    df = df[~df[col_name].isin(less_freq_ids)]

    # Return
    return df, num_less_freq_ids


In [ ]:
col_name = 'registrationID'
freq = 4

# Drop less frequent GPS columns
kelowna_gps_01Aug, num_thrice_occurrence_ids = drop_less_freq_ids(kelowna_gps_01Aug, col_name, freq)

# Calculate the number of such IDs
print(f"Number of registrationIDs occurring less than four times: {num_thrice_occurrence_ids}")


In [ ]:
def drop_duplicate_gps(df, gps_col_name, date_col_name):
    # Drop duplicates
    df = df.drop_duplicates()
    df[date_col_name] = pd.to_datetime(df[date_col_name])

    # Shape before dropping duplicates
    rows_before = df.shape[0]

    # Drop rows with duplicate registrationID and DateTime
    df = df.drop_duplicates(subset=[gps_col_name, date_col_name])

    # Shape after dropping duplicates
    rows_after = df.shape[0]

    # Calculate and print the number of rows dropped
    rows_dropped = rows_before - rows_after
    
    # Display
    return df, rows_dropped
    

In [ ]:

# Column name and count limit
gps_col_name = 'registrationID'
date_col_name = 'DateTime'

# Drop duplicates
kelowna_gps_01Aug, rows_dropped = drop_duplicate_gps(kelowna_gps_01Aug, gps_col_name, date_col_name)

print(f"Number of rows dropped: {rows_dropped}")
display(kelowna_gps_01Aug)

In [ ]:
# Calculate the start and end times
start_time_k = kelowna_gps_01Aug['DateTime'].min()
end_time_k = kelowna_gps_01Aug['DateTime'].max()

print(f"kelowna_gps_01Aug data was collected from {start_time_k} to {end_time_k}.")

In [ ]:
def display_shp_in_contextily_v1(path, crs, dim):
    # Read shapefile
    shp = gpd.read_file(path)
    
    # Set the Coordinate Reference System (CRS) if not already set (WGS84 - EPSG:4326 for lat/long)
    shp.to_crs(epsg=crs, inplace=True)
    
    # Plot the geometries on a basemap
    fig, ax = plt.subplots(figsize=(dim, dim))
    
    # Plot the roads (your data)
    shp.plot(ax=ax, color='blue', linewidth=1, alpha=0.3)
    
    # Use OpenStreetMap for the basemap
    ctx.add_basemap(ax, crs=shp.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
    
    # Show the plot
    plt.show()

    return 0


In [ ]:

# Constant: SHP path and CRS value
path_to_shp = 'C:/data/boundaries_v2/inner_boundary_Kelowna.shp'
epsg = 4326

display_shp_in_contextily_v1(path_to_shp, epsg, 6)


In [ ]:
def convert_roads_geojson_to_osmnx_graphml_v1(path, crs_no):

    # Load the GeoJSON file
    roads_gdf = gpd.read_file(path)
    
    # Ensure CRS is WGS84 (EPSG:4326)
    roads_gdf = roads_gdf.to_crs(epsg=crs_no)
    
    # Initialize a directed graph
    roads_graphml = nx.MultiDiGraph()
    
    # Extract nodes and edges
    node_id = 0  # To uniquely identify nodes
    node_map = {}  # To store unique nodes and their IDs
    
    for _, row in roads_gdf.iterrows():
        if isinstance(row.geometry, LineString):
            # Extract coordinates from the LineString
            coords = list(row.geometry.coords)
            
            # Add nodes and edges to the graph
            for i in range(len(coords) - 1):
                u, v = coords[i], coords[i + 1]
                
                # Add nodes (check if they already exist)
                if u not in node_map:
                    node_map[u] = node_id
                    roads_graphml.add_node(node_id, x=u[0], y=u[1])
                    node_id += 1
                if v not in node_map:
                    node_map[v] = node_id
                    roads_graphml.add_node(node_id, x=v[0], y=v[1])
                    node_id += 1
                
                # Add edge with attributes
                roads_graphml.add_edge(node_map[u], node_map[v], length=LineString([u, v]).length)
        else:
            print(f"Skipping non-LineString geometry at index {_}")

    if crs_no == 4326:
        # Add CRS to the graph
        roads_graphml.graph['crs'] = "EPSG:4326"
    else:
        print('The crs_no is not 4326; skipping graph[crs] addition')

    return roads_graphml


In [ ]:
import geopandas as gpd
import networkx as nx
import osmnx as ox

# Constants
path = "C:/data/gps_sai/bc_gis/kelowna_roads_1.geojson"
crs = 4326

# Convert to graphml
roads_kelowna_json = convert_roads_geojson_to_osmnx_graphml_v1(path, crs)

# Save the graph as a GraphML file (optional)
ox.save_graphml(roads_kelowna_json, "C:/data/gps_sai/bc_gis/kelowna_roads_1.graphml")

# Visualize the graph (optional)
ox.plot_graph(roads_kelowna_json, node_size=5, edge_linewidth=0.5)


In [ ]:
# Importing road network from pytrack's graph module
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
import geopandas as gpd
from shapely.geometry import Point, LineString

from cybstr.pytrack.graph import graph, distance
from cybstr.pytrack.analytics import visualization
from cybstr.pytrack.matching import candidate, mpmatching_utils, mpmatching

# Get Kelowna boundary shapefile
kelowna_boundary = gpd.read_file('C:/data/boundaries_v2/inner_boundary_Kelowna.shp')

# Set the Coordinate Reference System (CRS) if not already set (WGS84 - EPSG:4326 for lat/long)
kelowna_boundary.to_crs(epsg=4326, inplace=True)

# Get the bounding box again
minx, miny, maxx, maxy = kelowna_boundary.total_bounds

# Convert the graph to GeoDataFrames
nodes, edges = ox.graph_to_gdfs(roads_kelowna_json)

latitude = kelowna_gps_01Aug["latitude"].to_list()
longitude = kelowna_gps_01Aug["longitude"].to_list()

# Convert to north, south, east, and west
north = maxy  # Maximum latitude
south = miny  # Minimum latitude
east = maxx   # Maximum longitude
west = minx   # Minimum longitude

points = [(lat, lon) for lat, lon in zip(latitude[:30], longitude[:30])]
roads_kelowna_json = graph.graph_from_bbox(*distance.enlarge_bbox(north, south, west, east, 500), simplify=True, network_type='drive')

In [ ]:
# Visualize the roads in pytrack's map module
loc = (np.mean(latitude[:30]), np.mean(longitude[:30]))
maps = visualization.Map(location=loc, zoom_start=15)
maps.add_graph(roads_kelowna_json, plot_nodes=True)
# maps

In [ ]:
def calculate_trip_id(df, gps_col_name, date_col_name, timelimt):

    # Convert DateTime column to datetime type
    df[date_col_name] = pd.to_datetime(df[date_col_name])
    
    # Sort by registrationID and DateTime to ensure proper order
    df = df.sort_values(by=[gps_col_name, date_col_name])
    
    # Calculate time difference in seconds between consecutive rows
    df.loc[:,"time_diff"] = df[date_col_name].diff().dt.total_seconds()
    
    # Assign trip_id based on the desired time gap
    df.loc[:,"trip_id"] = (df["time_diff"] > timelimt).cumsum() + 1
    
    # Fill NaN for the first row's time_diff
    df.loc[:,"time_diff"] = df["time_diff"].fillna(0)

    # Return the df
    return df


In [ ]:
# Assign trip_ids based on 1hr gap between consecutive GPS points
# Constants: column names and time limit trip
gps_col_name = "registrationID"
date_col_name = "DateTime"
timelimt = 3600

# Return and print the df
kelowna_gps_01Aug_id1 = calculate_trip_id(kelowna_gps_01Aug_id1, gps_col_name, date_col_name, timelimt)
display(kelowna_gps_01Aug_id1.head())

In [ ]:
# Ensure geometry column contains Point objects
import geopandas as gpd

kelowna_gps_01Aug_id1 = gpd.GeoDataFrame(kelowna_gps_01Aug_id1, geometry=gpd.points_from_xy(kelowna_gps_01Aug_id1.longitude, kelowna_gps_01Aug_id1.latitude), crs="EPSG:4326")
kelowna_gps_01Aug_id1 = kelowna_gps_01Aug_id1.reset_index()

# Check if the geometry column is valid
if isinstance(kelowna_gps_01Aug_id1, gpd.GeoDataFrame):
    # Extract x (longitude) and y (latitude) from the geometry column
    kelowna_gps_01Aug_id1["x"] = kelowna_gps_01Aug_id1.geometry.x
    kelowna_gps_01Aug_id1["y"] = kelowna_gps_01Aug_id1.geometry.y
else:
    raise ValueError("kelowna_gps_01Aug_id1 must be a GeoDataFrame with a valid geometry column.")

display(kelowna_gps_01Aug_id1.head())

In [ ]:
kelowna_gps_01Aug_id1_t1 = kelowna_gps_01Aug_id1[kelowna_gps_01Aug_id1['trip_id'] == 1] # Test: use trip_id 1

display(kelowna_gps_01Aug_id1_t1)

In [ ]:
# Initialize maps
# Retrieve all lat/long and make a list
lati_01Aug_id1_t1 = kelowna_gps_01Aug_id1_t1["latitude"].to_list()
long_01Aug_id1_t1 = kelowna_gps_01Aug_id1_t1["longitude"].to_list()

points_01Aug_id1_t1 = [(lat, lon) for lat, lon in zip(lati_01Aug_id1_t1[:30], long_01Aug_id1_t1[:30])]

# Center map around mean latitude and longitude
loc_01Aug_id1_t1 = (np.mean(lati_01Aug_id1_t1[:30]), np.mean(long_01Aug_id1_t1[:30]))
maps_01Aug_id1_t1 = visualization.Map(location=loc_01Aug_id1_t1, zoom_start=15)
#maps_01Aug_id1_t1

In [ ]:
# Function to get connected subgraphs

def connected_component_subgraphs(G):
    for c in nx.weakly_connected_components(G):
        yield G.subgraph(c)

# Take the largest connected subgraph
sgs = list(connected_component_subgraphs(roads_kelowna_json))
i = np.argmax([len(sg) for sg in sgs])
sg = sgs[i]
len(sg)


<H3>Trajectory for all IDs in a day</H3>

In [ ]:
# Constants: column names and time limit trip
gps_col_name = "registrationID"
date_col_name = "DateTime"
timelimt = 3600

# Return and print the df
kelowna_gps_01Aug = calculate_trip_id(kelowna_gps_01Aug, gps_col_name, date_col_name, timelimt)
display(kelowna_gps_01Aug.head())


In [ ]:
# Ensure geometry column contains Point objects
import geopandas as gpd

kelowna_gps_01Aug = gp.GeoDataFrame(kelowna_gps_01Aug, geometry=gp.points_from_xy(kelowna_gps_01Aug.longitude, kelowna_gps_01Aug.latitude), crs="EPSG:4326")
kelowna_gps_01Aug = kelowna_gps_01Aug.reset_index()

# Check if the geometry column is valid
if isinstance(kelowna_gps_01Aug, gpd.GeoDataFrame):
    # Extract x (longitude) and y (latitude) from the geometry column
    kelowna_gps_01Aug["x"] = kelowna_gps_01Aug.geometry.x
    kelowna_gps_01Aug["y"] = kelowna_gps_01Aug.geometry.y
else:
    raise ValueError("kelowna_gps_01Aug must be a GeoDataFrame with a valid geometry column.")

display(kelowna_gps_01Aug.head())

In [ ]:
# Retrieve all lat/long and make a list
latitude_kelowna_gps_01Aug = kelowna_gps_01Aug["latitude"].to_list()
longitude_kelowna_gps_01Aug = kelowna_gps_01Aug["longitude"].to_list()

points_kelowna_gps_01Aug = [(lat, lon) for lat, lon in zip(latitude_kelowna_gps_01Aug[:30], longitude_kelowna_gps_01Aug[:30])]

# Center map around mean latitude and longitude
loc = (np.mean(latitude_kelowna_gps_01Aug[:30]), np.mean(longitude_kelowna_gps_01Aug[:30]))
maps = visualization.Map(location=loc, zoom_start=15)
#maps

In [ ]:

failed_trips = []

# Call the function to map-match and create a GeoDataFrame
matched_trips_gdf, failed_trips = map_match_and_return_geodataframe_v3(
    df=kelowna_gps_01Aug,
    log_name="kelowna_01Aug",
    graph=roads_kelowna_json, 
    day_col="DateTime", 
    gps_id_col="registrationID", 
    trip_id_col="trip_id", 
    interp_dist=50, 
    radius=90, 
    max_trips=10
)


In [ ]:
matched_trips_gdf

In [ ]:
# Save/load the trips dataframe
'''
matched_trips_gdf.to_csv('matched_trips_gdf.csv')
matched_trips_gdf.head()
'''
'''
matched_trips_df = df.read_csv()
# convert to gdf?
'''

In [ ]:
# Visualize the trips in folium
###############################

folium_map = visualize_trips_in_folium(
    trips_gdf=matched_trips_gdf,  # GeoDataFrame from the map-matching function
    trip_id_col="trip_id"         # Column containing trip IDs
)

# Save and display the map
folium_map.save("matched_trips_folium_map_1000.html")

In [ ]:

import contextily as ctx

path = 'C:/data/gps_sai/bc_gis/junctions/Kelowna_Water_Harvey_Pandosy_Junc.shp'

shp = gpd.read_file(path)
    
# Set the Coordinate Reference System (CRS) if not already set (WGS84 - EPSG:4326 for lat/long)
shp.to_crs(epsg=4326, inplace=True)
    
# Plot the geometries on a basemap
fig, ax = plt.subplots(figsize=(6, 6))
    
# Plot the roads (your data)
shp.plot(ax=ax, color='blue', linewidth=1, alpha=0.3)
    
# Use OpenStreetMap for the basemap
ctx.add_basemap(ax, crs=shp.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
    
# Show the plot
plt.show()

In [ ]:
# Find out how many linestrings from the trajectory gdb is crossing the junction

# Perform the spatial join to find intersecting linestrings
intersecting_trajectories = gpd.sjoin(matched_trips_gdf, shp, how="inner", predicate="intersects")

# Remove duplicates to get unique linestrings that pass through intersections
unique_intersecting_trajectories = intersecting_trajectories.drop_duplicates(subset=["gps_id", "trip_id"])

# Count the Number of Intersecting Trajectories
num_intersecting_trajectories = unique_intersecting_trajectories.shape[0]
print(f"Number of trajectories passing through intersections: {num_intersecting_trajectories}")


In [ ]:
# Visualize the trips in folium

folium_map = visualize_trips_in_folium(
    trips_gdf=unique_intersecting_trajectories,  # GeoDataFrame from the map-matching function
    trip_id_col="trip_id"         # Column containing trip IDs
)

# Save and display the map
folium_map.save("matched_trips_folium_map_1000_unique_intersecting_trajectories.html")